In [45]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [46]:
from tensorflow import keras
from keras.models import Sequential 
import cv2
import os
from pathlib import Path
# import face_recognition
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from skimage import io
import glob
import random
from sklearn.model_selection import train_test_split



In [47]:
from operator import concat
#Define Paths
data_train_path = Path('/content/drive/My Drive/data/FaceData/train')         # Define path for data
data_test_path = Path('/content/drive/My Drive/data/FaceData/test')         # Define path for data


def path_produce(premiere_path,emotion_path):
  y = premiere_path/emotion_path
  return y
emotion = ["angry", "disgusted", "fearful", "happy","neutral", "sad", "suprised"]
emotion_number = ["0", "1", "2", "3","4", "5", "6"]
# emotion_path = ["angry_train_path", "disgusted_train_path", "fearful_train_path", "happy_train_path", "neutral_train_path", "sad_train_path", "suprised_train_path"]


def get_images(images_path): 
  return images_path.glob('*.png')

# all_train_data = pd.DataFrame(columns=['image' , 'class'],index=None)
# all_test_data  = pd.DataFrame(columns=['image' , 'class'],index=None)



def get_path_give_images(path,emotion,emotion_number):
  all_data = pd.DataFrame(columns=['image' , 'emotionclass'],index=None)
  j=0;
  for i in emotion:
    EmPath = path_produce(path, i)
    array_images  = get_images(EmPath) 
    data = pd.DataFrame(array_images, columns=['image'],index=None)
    data['emotionclass'] = emotion_number[j]
    j+=1
    d=[all_data,data]
    all_data = pd.concat(d)
  return all_data

all_train_data = get_path_give_images(data_train_path,emotion,emotion_number)
all_test_data = get_path_give_images(data_test_path,emotion,emotion_number)

train_data = all_train_data.sample(frac=1.).reset_index(drop=True)
test_data = all_test_data.sample(frac=1.).reset_index(drop=True)

X = train_data.image;
y = train_data.emotionclass;


X_val, X_train, y_val , y_train = train_test_split(X, y, test_size=0.9, random_state=1)
df1 = {'image': X_train, 'emotionclass': y_train}
train_data = pd.DataFrame(df1,index=None)

df2 = {'image': X_val, 'emotionclass': y_val}
validation_data = pd.DataFrame(df2,index=None)



KeyboardInterrupt: ignored

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt


In [48]:
validation_data.head()

,image,emotionclass
2148,/content/drive/My Drive/data/FaceData/train/ha...,3
11128,/content/drive/My Drive/data/FaceData/train/ha...,3
1212,/content/drive/My Drive/data/FaceData/train/ha...,3
16303,/content/drive/My Drive/data/FaceData/train/ne...,4
25123,/content/drive/My Drive/data/FaceData/train/ne...,4


In [49]:
# # plot intensity of pixels of 5 images
# for i in range(3):
#   plt.xlabel("Pixel values")
#   plt.ylabel("Intensity distribution of an image")
#   plt.title("Image{}".format(i+1))
#   a = train_data['image'].iloc[i]
#   img = io.imread(a,1)
#   plt.hist(img.ravel())
#   plt.show()

In [50]:
# Parameters
IMG_SIZE = 256  # Image size
BATCH_SIZE = 64

# Preprocessing object
idg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

# Turn values to strings as a requirement for flow_from_dataframe
train_data['image'] = train_data['image'].astype("string")
validation_data['image'] = validation_data['image'].astype("string")
test_data['image'] = test_data['image'].astype("string")

# Create train generator
train_gen = idg.flow_from_dataframe(dataframe= train_data,
                                         x_col = 'image',
                                         y_col = 'emotionclass',
                                         target_size = (48,48),
                                         class_mode = 'categorical',
                                         color_mode = 'grayscale',
                                         batch_size = BATCH_SIZE,
                                         )

# Create test generator
validation_gen = idg.flow_from_dataframe(dataframe= validation_data, 
                                         x_col = 'image',
                                         y_col = 'emotionclass',
                                         target_size = (48,48),
                                         class_mode = 'categorical',
                                         color_mode = 'grayscale',
                                         batch_size = BATCH_SIZE,
                                         )
# Create test generator
test_gen = idg.flow_from_dataframe(dataframe= test_data, 
                                         x_col = 'image',
                                         y_col = 'emotionclass',
                                         target_size = (48,48),
                                         class_mode = 'categorical',
                                         color_mode = 'grayscale',
                                         batch_size = BATCH_SIZE,
                                         )

Found 22985 validated image filenames belonging to 6 classes.
Found 2553 validated image filenames belonging to 6 classes.
Found 6347 validated image filenames belonging to 6 classes.


In [51]:
# # Parameters
# IMG_SIZE = 256  # Image size
# BATCH_SIZE = 16

# # Preprocessing object
# idg = tf.keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False, samplewise_center=False,
#     featurewise_std_normalization=False, samplewise_std_normalization=False,
#     zca_whitening=False, zca_epsilon=1e-06, rotation_range=10, width_shift_range=0.1,
#     height_shift_range=0.1, brightness_range=None, shear_range=0.1, zoom_range=0.1,
#     channel_shift_range=0.0, fill_mode='nearest', cval=0.0, horizontal_flip=True,
#     vertical_flip=True, rescale=1/255.0, preprocessing_function=None,
#     data_format=None, validation_split=0.0, dtype=None
# )

# # Turn values to strings as a requirement for flow_from_dataframe
# train_data['image'] = train_data['image'].astype("string")
# validation_data['image'] = validation_data['image'].astype("string")
# test_data['image'] = test_data['image'].astype("string")

# # Create train generator
# train_gen = idg.flow_from_dataframe(dataframe= train_data,
#                                          directory=None, 
#                                          x_col = 'image',
#                                          y_col = 'emotionclass',
#                                          class_mode = 'categorical',
#                                          color_mode = 'grayscale',
#                                          target_size = (IMG_SIZE,IMG_SIZE), 
#                                          batch_size = BATCH_SIZE,
#                                          shuffle = True,
#                                          )

# # Create test generator
# validation_gen = idg.flow_from_dataframe(dataframe= validation_data, 
#                                          directory=None, 
#                                          x_col = 'image',
#                                          y_col = 'emotionclass',
#                                          class_mode = 'sparse',
#                                          color_mode = 'grayscale',
#                                          target_size = (IMG_SIZE,IMG_SIZE), 
#                                          batch_size = BATCH_SIZE,
#                                          shuffle = True,

#                                          )

# # Create test generator
# test_gen = idg.flow_from_dataframe(dataframe= test_data, 
#                                          directory=None, 
#                                          x_col = 'image',
#                                          y_col = 'emotionclass',
#                                          class_mode = 'categorical',
#                                          color_mode = 'grayscale',
#                                          target_size = (IMG_SIZE,IMG_SIZE), 
#                                          batch_size = BATCH_SIZE,
#                                          shuffle = False,
#                                          )

In [52]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
import tensorflow as tf


emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(6, activation='softmax'))

In [53]:
emotion_model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
#Train Model
# from keras.models import load_model

# TRAINING = True
NUM_EPOCHS = 20
# model_path = '/content/drive/MyDrive/Project_Colab/Pneumonia_detection/my_model.h5'

# if TRAINING:
# emotion_model_info = emotion_model.fit_generator(train_gen, 
#                                         epochs=NUM_EPOCHS, 
#                                         steps_per_epoch=None,
#                                         validation_data=validation_gen,
#                                         validation_steps=len(validation_gen))

emotion_model_info = emotion_model.fit_generator(
        train_gen,
        steps_per_epoch=len(train_gen) // 5,
        epochs=20,
        validation_data=validation_gen,
        validation_steps=len(validation_gen) // 5)
   
# else:

  # returns a compiled model
  # identical to the previous one
  # model = load_model(model_path, compile=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/20
43/72 [================>.............] - ETA: 5:26 - loss: 1.6878 - accuracy: 0.2609

In [ ]:
#Train Model
from keras.models import load_model

TRAINING = True
NUM_EPOCHS = 30
model_path = '/content/drive/MyDrive/Project_Colab/Pneumonia_detection/my_model.h5'

if TRAINING:
  r =emotion_model.fit(train_gen, 
                epochs=NUM_EPOCHS, 
                validation_data=validation_gen,
                validation_steps=len(validation_gen),
                steps_per_epoch=None)
   
  emotion_model.save(model_path)

else:

  # returns a compiled model
  # identical to the previous one
  emorion_model = load_model(model_path, compile=True)